# AI Assisted Executive Order Analysis

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import csv
from openai import OpenAI
import my_secrets

In [ ]:
# Set your OpenAI API key

client = OpenAI(api_key=my_secrets.OPENAI_API_KEY)

In [ ]:
# Load the input CSV, scripts that generated this document are in the Initial_EO_Research directory
input_file = 'federal_register_eos.csv'
df = pd.read_csv(input_file)

## AI Analysis

In my initial tests this block ran for 18 minutes. Going to check into rate limiting. Also, for some reason the results were sporadially incomplete. This can do with some tweaking.

<div class="alert alert-block alert-warning">

The code in the block below **WILL COST YOU MONEY**!

</div>

In [7]:
results = []

def analyze_text(text):
    prompt = (
        "Read the following Executive Order text and return two things:\n"
        "1. A one or two word sentiment (like 'positive', 'neutral', 'urgent')\n"
        "2. A brief summary in one or two sentences.\n\n"
        f"Text:\n{text[:3000]}\n\n"  # Truncate to stay within limits
        "Return result as:\nSentiment: <sentiment>\nSummary: <summary>"
    )

    try:
        response = client.chat.completions.create(
            model="gpt-4",
            messages=[{"role": "user", "content": prompt}],
            temperature=0.5,
            max_tokens=200
        )

        output = response.choices[0].message.content
        sentiment = ""
        summary = ""
        for line in output.splitlines():
            if line.lower().startswith("sentiment:"):
                sentiment = line.split(":", 1)[1].strip()
            elif line.lower().startswith("summary:"):
                summary = line.split(":", 1)[1].strip()
        return sentiment, summary

    except Exception as e:
        print(f"Error processing text: {e}")
        return "Error", "Could not process content"

# Loop through each EO URL
for index, row in df.iterrows():
    eo_number = row['EO #']
    url = row['URL']
    try:
        response = requests.get(url, timeout=10)
        soup = BeautifulSoup(response.content, 'html.parser')

        # Extract EO content
        content_div = soup.find('div', {'class': 'abstract'}) or soup.find('div', {'id': 'document'})
        text = content_div.get_text(strip=True, separator=' ') if content_div else ''

        # Use OpenAI to get sentiment and summary
        sentiment, summary = analyze_text(text[:4000])  # Truncate to fit prompt size limit

        results.append({
            'EO Number': eo_number,
            'Content Sentiment': sentiment,
            'Summary': summary
        })

    except Exception as e:
        results.append({
            'EO Number': eo_number,
            'Content Sentiment': 'error',
            'Summary': f'Failed to process: {e}'
        })

KeyboardInterrupt: 

### Export Results to CSV

In [ ]:
output_file = 'eo_summary_output.csv'
with open(output_file, 'w', newline='', encoding='utf-8') as f:
    writer = csv.DictWriter(f, fieldnames=['EO Number', 'Content Sentiment', 'Summary'])
    writer.writeheader()
    writer.writerows(results)

print(f"Output written to {output_file}")

Output written to eo_summary_output.csv
